In [1]:
from mccd.random_clifford_circuit import *
from surface_sim.setups import CircuitNoiseSetup
from surface_sim.models import CircuitNoiseModel
from surface_sim import Detectors
from surface_sim.experiments import schedule_from_circuit, experiment_from_schedule
import time
import stim

from pathlib import Path
import stim
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from joblib import Parallel, delayed

from pymatching import Matching as MWPM
from mle_decoder import MLEDecoder as MLE
from stimbposd import BPOSD
from sklearn.metrics import accuracy_score

METHOD_MAP = {
    'BPOSD': BPOSD,
    'MLE': MLE,
    'MWPM': MWPM,
}

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [18]:
from surface_sim.circuit_blocks.rot_surface_code_css import gate_to_iterator
print('Rotated', gate_to_iterator.keys())
from surface_sim.circuit_blocks.unrot_surface_code_css import gate_to_iterator
print('Unrotated', gate_to_iterator.keys())
ROT_GATES = list('IXZ')
UNR_GATES = list('IHXZ')
MCCD_GATES = ['I', 'X', 'Y', 'Z', 'H']

Rotated dict_keys(['TICK', 'I', 'S', 'X', 'Z', 'CX', 'CNOT', 'R', 'RZ', 'RX', 'M', 'MZ', 'MX'])
Unrotated dict_keys(['TICK', 'I', 'S', 'H', 'X', 'Z', 'CX', 'CNOT', 'R', 'RZ', 'RX', 'M', 'MZ', 'MX'])


In [3]:
def to_stim_circuit(mccd_circuit):
    res = stim.Circuit()
    # Must have R and M. Error inactive layout.
    for n in range(mccd_circuit.n_logical_qubits):
        res.append('R', [n])

    for name, timestep, qubits in mccd_circuit:
        res.append(name, qubits)

    for n in range(mccd_circuit.n_logical_qubits):
        res.append('M', [n])

    return res

def print_random_circuit(c: RandomCliffordCircuit):
    return list(c)


def run_decoder(name: str, circuit: stim.Circuit, shots: int):
    method = METHOD_MAP[name](circuit.detector_error_model())
    syndrome, labels = sampler.sample(shots=shots, separate_observables=True)
    begin = time.time_ns()
    predictions = method.decode_batch(syndrome)
    end = time.time_ns()
    logical_accuracy = accuracy_score(labels, predictions)
    walltime_seconds = (end - begin) / 1e9
    return dict(
        decoder=name,
        logical_accuracy=logical_accuracy,
        walltime_seconds=walltime_seconds,
    )

def compile_to_physical(log_cir: stim.Circuit, distance: int, noise_prob: float, rotated=True) -> stim.Circuit:
    if rotated:
        from surface_sim.circuit_blocks.rot_surface_code_css import gate_to_iterator
        from surface_sim.layouts import rot_surface_codes
        layouts = rot_surface_codes(log_cir.num_qubits, distance=distance)
    else:
        from surface_sim.circuit_blocks.unrot_surface_code_css import gate_to_iterator
        from surface_sim.layouts import unrot_surface_codes
        layouts = unrot_surface_codes(log_cir.num_qubits, distance=distance)

    setup = CircuitNoiseSetup()
    model = CircuitNoiseModel.from_layouts(setup, *layouts)
    detectors = Detectors.from_layouts("pre-gate", *layouts)
    setup.set_var_param("prob", noise_prob)
    schedule = schedule_from_circuit(log_cir, layouts, gate_to_iterator)
    phy_cir: stim.Circuit = experiment_from_schedule(
        schedule, model, detectors, anc_reset=True
    )
    return phy_cir


def random_mirror_symmetric_clifford(n_logical_qubits: int, circuit_index: str, depth: int, rotated=True) -> stim.Circuit:
    logical_class = TypeICircuit if circuit_index == '3' else TypeIICircuit
    random_circuit = logical_class(
        n_logical_qubits=n_logical_qubits,
        circuit_index=circuit_index,
        depth=depth,
        single_qubit_gate_list=ROT_GATES if rotated else ROT_GATES,
    )
    random_circuit.sample_circuit()
    return to_stim_circuit(random_circuit)



## Circuit Depths

Finally, to ensure that MCCD’s training generalizes to new circuits, we train MCCD with data from logical circuit depth D = 2, 4, 6, 8 and 10 (D = 4, 8, 12, 16 and 20) and test the model on a wide range of unseen circuits with depth up to D = 18 (D = 36) for Circuit Type I (Circuit Type II), defined below.

In [7]:
n_logical_qubits=2
circuit_index = '3'
depth = 2
code_distance=3 # 2 is not ok with rot code.
noise_prob = 1e-3
num_shots = 1000
num_circuits = 100


In [8]:
logical_class = TypeICircuit if circuit_index == '3' else TypeIICircuit

random_circuit = logical_class(
    n_logical_qubits=n_logical_qubits,
    circuit_index=circuit_index,
    depth=depth,
    single_qubit_gate_list=ROT_GATES)

random_circuit.sample_circuit()

In [9]:
random_circuit.to_string_lines()

['ZZ', 'ZI']

In [10]:
print_random_circuit(random_circuit)

[(np.str_('Z'), 0, [0]),
 (np.str_('Z'), 0, [1]),
 (np.str_('Z'), 1, [0]),
 (np.str_('I'), 1, [1]),
 (np.str_('Z'), 1, [0]),
 (np.str_('I'), 1, [1]),
 (np.str_('Z'), 0, [0]),
 (np.str_('Z'), 0, [1])]

In [11]:
random_circuit.n_logical_qubits

2

In [13]:
log_cir = to_stim_circuit(random_circuit)

In [67]:
print(log_cir)

R 0 1
I 0
Z 1
I 0 1 0 1 0
Z 1
M 0 1


In [14]:
from surface_sim.layouts import rot_surface_codes

layouts = rot_surface_codes(log_cir.num_qubits, distance=code_distance)
setup = CircuitNoiseSetup()
model = CircuitNoiseModel.from_layouts(setup, *layouts)
detectors = Detectors.from_layouts("pre-gate", *layouts)

setup.set_var_param("prob", 1e-3)

schedule = schedule_from_circuit(log_cir, layouts, gate_to_iterator)
phy_cir: stim.Circuit = experiment_from_schedule(
    schedule, model, detectors, anc_reset=True
)

In [15]:
print(phy_cir)

QUBIT_COORDS(0, 0) 0
QUBIT_COORDS(0, 2) 1
QUBIT_COORDS(0, 4) 2
QUBIT_COORDS(2, 0) 3
QUBIT_COORDS(2, 2) 4
QUBIT_COORDS(2, 4) 5
QUBIT_COORDS(4, 0) 6
QUBIT_COORDS(4, 2) 7
QUBIT_COORDS(4, 4) 8
QUBIT_COORDS(-1, 1) 9
QUBIT_COORDS(1, 3) 10
QUBIT_COORDS(3, 1) 11
QUBIT_COORDS(5, 3) 12
QUBIT_COORDS(1, 1) 13
QUBIT_COORDS(1, 5) 14
QUBIT_COORDS(3, -1) 15
QUBIT_COORDS(3, 3) 16
QUBIT_COORDS(0, 8) 17
QUBIT_COORDS(0, 10) 18
QUBIT_COORDS(0, 12) 19
QUBIT_COORDS(2, 8) 20
QUBIT_COORDS(2, 10) 21
QUBIT_COORDS(2, 12) 22
QUBIT_COORDS(4, 8) 23
QUBIT_COORDS(4, 10) 24
QUBIT_COORDS(4, 12) 25
QUBIT_COORDS(-1, 9) 26
QUBIT_COORDS(1, 11) 27
QUBIT_COORDS(3, 9) 28
QUBIT_COORDS(5, 11) 29
QUBIT_COORDS(1, 9) 30
QUBIT_COORDS(1, 13) 31
QUBIT_COORDS(3, 7) 32
QUBIT_COORDS(3, 11) 33
R 0 1 2 3 4 5 6 7 8 17 18 19 20 21 22 23 24 25
X_ERROR(0.001) 0 1 2 3 4 5 6 7 8 17 18 19 20 21 22 23 24 25
I 9 10 11 12 13 14 15 16 26 27 28 29 30 31 32 33
DEPOLARIZE1(0.001) 9 10 11 12 13 14 15 16 26 27 28 29 30 31 32 33
TICK
I 5 13 7 10 1 15 4 0 1

In [16]:
sampler = phy_cir.compile_detector_sampler()

In [22]:
n_logical_qubits=2
circuit_index = '4'
depth = 36
code_distance=3 # 2 is not ok with rot code.
noise_prob = 1e-3
num_shots = 1000
num_circuits = 10

for i in range(num_circuits):
    log_cir = random_mirror_symmetric_clifford(n_logical_qubits, circuit_index, depth)
    phy_cir = compile_to_physical(log_cir, code_distance, noise_prob)
    for name in METHOD_MAP.keys():
        res = run_decoder(name, phy_cir, shots=num_shots)
        print(res)


{'decoder': 'BPOSD', 'logical_accuracy': 0.997, 'walltime_seconds': 0.003882}
{'decoder': 'MLE', 'logical_accuracy': 0.999, 'walltime_seconds': 0.182786}
{'decoder': 'MWPM', 'logical_accuracy': 0.999, 'walltime_seconds': 9.3e-05}
{'decoder': 'BPOSD', 'logical_accuracy': 0.998, 'walltime_seconds': 0.004594}
{'decoder': 'MLE', 'logical_accuracy': 0.999, 'walltime_seconds': 0.19865}
{'decoder': 'MWPM', 'logical_accuracy': 0.999, 'walltime_seconds': 5.2e-05}
{'decoder': 'BPOSD', 'logical_accuracy': 0.998, 'walltime_seconds': 0.004971}
{'decoder': 'MLE', 'logical_accuracy': 1.0, 'walltime_seconds': 0.230359}
{'decoder': 'MWPM', 'logical_accuracy': 0.999, 'walltime_seconds': 5.4e-05}
{'decoder': 'BPOSD', 'logical_accuracy': 1.0, 'walltime_seconds': 0.004142}
{'decoder': 'MLE', 'logical_accuracy': 0.999, 'walltime_seconds': 0.179817}
{'decoder': 'MWPM', 'logical_accuracy': 0.998, 'walltime_seconds': 4.8e-05}
{'decoder': 'BPOSD', 'logical_accuracy': 0.999, 'walltime_seconds': 0.004808}
{'decod